List of things to do:

- Gather
- Scatter
- Stencil
- Transpose

A thread Block programming example to show that it's not predefined the way the threads are launched

# El modelo de programación en paralelo

El lector podrá ahora darse cuenta por sí mismo que hay diferencias al programar en paralelo, con respecto a la forma usual. La principal de ellas es que el GPU no es un procesador serial, sino un procesador *stream*. Un procesador serial, también conocido como arquitectura de von Neumann, ejecuta instrucciones serialmente, actualizando la memoria a medida que avanza. Un procesador en *stream* funciona de una manera ligeramente diferente, ejecutando una función (tal como un fragmento de programa) en un conjunto de registros de entrada, produciendo un conjunto de registros de salida (e.g. pixeles sombreados) en paralelo. Los procesadores en stream, típicamente se refieren a dicha función como *kernel* y al conjunto de registros como *stream* (muy imaginativos). La información es lanzada al procesador, operada vía una función *kernel*, y sacada a la memoria, como se muestra en la figura 1. Cada elemento pasado al procesador es procesado independientemente, sin ser influido por los demás elementos. Esto permite a la arquitectura ejecutar un programa en paralelo sin la necesidad de exponer las unidades paralelas o cualquier construcción en paralelo al programador. Es por dicha razón que al escribir la función kernel en el capítulo pasado no notamos el paralelismo, no fue sino hasta el momento de lanzar el kernel que vimos esto.

![](http://http.developer.nvidia.com/GPUGems/elementLinks/fig37-01.jpg)

<h6 align="center">Figura 1: Modelo de ejecución de un programa en el GPU (imagen tomada de los GPUGems de NVIDIA) </h6>

Y qué tal si quisiésemos hacer física con esto, sería increíblemente incómodo que el único tipo de dato que pudiésemos manejar fuera el sombreado de un pixel; sin embargo, gracias a los avances en GPU se pueden manejar registros que no necesariamente son pixeles y sombreados; bien podría ser un conjunto arbitrario de datos, por ejemplo puntos de una red y ecuaciones fisicas. Digamos, que queremos implementar una simulación simple de partículas libres y dejar que el GPU lleve a cabo la mayor parte de las operaciones físicas. Usando texturas de punto flotante y buffers de pixel (pbuffers), guardamos la posición de las partículas, valocidades, y orientaciones. Un kernel hace los cálculos necesarios para obtener la nueva posición de cada partícula. Para calcular todas y cada una de las nuevas posiciones simplemente tomamos un cuadrante que tenga tantos pixeles como partículas, tal y como se muestra en la figura 2. Las coordenadas de textura para el cuadrante indican al kernel cuál de las partículas guardadas procesará. El resultado guardado en el pbuffer contiene los valores de las nuevas posiciones.

![](http://http.developer.nvidia.com/GPUGems/elementLinks/fig37-02.jpg)

<h6 align="center">Figura 2: Ejemplo de un sistema de partículas (imagen tomada de los GPUGems de NVIDIA) </h6>

Este ejemplo de un sistema de partículas demuestra una obvia pero útil aplicación del GPU para el cómputo de propósito general (GPGPU). La operación de actualizar la posición de las partículas puede generalizarse al proceso de aplicar una función a un arreglo. Esta operación - también llamada mapeo en lenguajes de programación funcional - puede ser usada para una amplia variedad de aplicaciones.

Pero **¿qué es este mapeo del que tanto hablamos?**

# Patrones de comunicación

## Mapeo

En el mapeo, una función opera en cada elemento de un arreglo de entrada y produce un resultado, por ejemplo sumar a cada entrada de un vector un número, o multiplicar todo el vector por una constante, con la conveniencia de que lo que se lee en la posición *i* de la entrada sirve para obtener el resultado que se escribe en la posición *i* de la salida. Para procesar todos los elementos de la entrada en paralelo, una función de mapeo debe ser pura. Esto quiere decir que genera exactamente el mismo resultado para la misma entrada, y que su ejecución no tiene efectos secundarios.

Dado que no hay necesidad de sincronizar a los threads, y no se comparte información, el patrón de mapeo se ajusta perfectamente a arquitecturas paralelas, de muchos núcleos. En las implementaciones en paralelo de los patrones de mapeo, cada thread ejecuta una instancia de una función mapeo y genera su correspondiente resultado. Éste patron es usado en muchos dominios, incluyendo el procesamiento de imágenes, simulaciones financieras, simulaciones físicas, etc.

A continuación mostramos un código para multiplicar un vector de 100 entradas (predefinidas del 1 al 100) por un escalar.

```C
#include <stdio.h>

#define ESCALAR 3

__global__ void multiplicar_por_escalar(float * escalar, float * d_salida, float * d_entrada){
	int idx = threadIdx.x;
    float f = d_entrada[idx];
    d_salida[idx] = f*escalar;
}

int main(int argc, char ** argv) {

	const int TAMANIO_ARREGLO = 100;
	const int BYTES_ARREGLO = TAMANIO_ARREGLO * sizeof(float);

	// Generamos el arreglo de entrada en el anfitrion
	float h_entrada[TAMANIO_ARREGLO];
    
	for (int i = 0; i < TAMANIO_ARREGLO; i++) {
		h_entrada[i] = float(i);
	}
    
	float h_salida[TAMANIO_ARREGLO];

	// Declaramos apuntadores de memoria en GPU
	float * d_entrada;
	float * d_salida;

	// Reservamos memoria del GPU
	cudaMalloc((void**) &d_entrada, BYTES_ARREGLO);
	cudaMalloc((void**) &d_salida, BYTES_ARREGLO);

	// Copiamos informacion al GPU
	cudaMemcpy(d_entrada, h_entrada, BYTES_ARREGLO, cudaMemcpyHostToDevice);

	// Lanza el kernel
	multiplicar_por_escalar<<<1, TAMANIO_ARREGLO>>>(ESCALAR, d_salida, d_entrada);

	// Copiamos el arreglo resultante al GPU
	cudaMemcpy(h_salida, d_salida, BYTES_ARREGLO, cudaMemcpyDeviceToHost);

	// Imprimimos el arreglo resultante
	for (int i =0; i < TAMANIO_ARREGLO; i++) {
		printf("%f \n", h_salida[i]);
	}

	cudaFree(d_entrada);
	cudaFree(d_salida);

	return 0;
}```

Si al lector se le hace conocido el código anterior es porque en realidad es el visto en el capítulo anterior, salvo algunas modificaciones pequeñas. **¿Las pueden encontrar?**

En este punto la pregunta en el aire debe ser **¿Qué demonios es `#define ESCALAR 3`?** Bueno, simplemente definimos una constante que se llama *ESCALAR* y que tiene como valor *3*.

Visto esto, se deja como ejercicio al lector hacer un código (en la celda de abajo) que sume 3 a cada una de las entradas de un vector (pueden usar el vector que quieran, en especial el que hemos venido utilizando).

## Dispersar y reunir

Los patrones dispersar (scatter) y reunir (gather) son similares a los patrones mapeo pero sus accesos a la memoria son aleatorios. Dispersar es una función mapeo que escribe en posiciones aleatorias, y reunir es la combinación de una función mapeo con accesos a memoria que leen de entradas aleatorias. Las implementacionesen paralelo de los patrones dispersar y reunir son similares a las implementaciones del mapeo. Dicho patrón se encuentra comunmente en aplicaciones estadísticas.

```C
#include <stdio.h>

#define NUM_BLOCKS 16
#define ANCHURA_BLOCK 1

__global__ void hola()
{
    printf("Hola mundo! Soy un thread en el bloque %d\n", blockIdx.x);
}


int main(int argc,char **argv)
{
    // lanzar el kernel
    hola<<<NUM_BLOCKS, ACHURA_BLOCK>>>();

    // forzar a los printf() para que se muestren
    cudaDeviceSynchronize();

    printf("Eso es todo amigos!\n");

    return 0;
}```

# Referencias

1. [Curso de Udacity - Intro to Parallel Programming](https://www.udacity.com/course/intro-to-parallel-programming--cs344)
2. [GPUGems - Capítulo 37](http://http.developer.nvidia.com/GPUGems/gpugems_ch37.html)
3. [Paraprox: Pattern-Based Approximation for Data Parallel Applications](http://cccp.eecs.umich.edu/papers/samadi-asplos14.pdf)

# Materiales adicionales

1. [GPUGems](http://developer.nvidia.com/object/gpu_gems_home.html)
2. [gpgpu.org](http://gpgpu.org/)